In [24]:
%load_ext autoreload
%autoreload 2

# Paramètres de la pipeline

In [1]:
code_departement = "093"
logs_level = "WARNING"

# Imports & setup

In [2]:
import duckdb
import geopandas as gpd
import pandas as pd

from potentiel_solaire.attach_buildings_to_schools import attach_buildings_to_schools
from potentiel_solaire.constants import DEFAULT_CRS, ALGORITHME_FOLDER, DATA_FOLDER
from potentiel_solaire.sources.bd_topo import extract_bd_topo, get_topo_zones_of_interest, \
    get_topo_buildings_of_interest
from potentiel_solaire.sources.bd_pci import extract_bd_pci
from potentiel_solaire.sources.bd_solar_irradiation import extract_bd_irradiation
from potentiel_solaire.sources.schools_establishments import extract_schools_establishments, \
    get_schools_establishments_of_interest
from potentiel_solaire.sources.protected_buildings import extract_protected_buildings, get_areas_with_protected_buildings
from potentiel_solaire.features.solar_potential import calculate_solar_potential
from potentiel_solaire.aggregate import aggregate_solar_potential_by
from potentiel_solaire.logger import get_logger

logger = get_logger()
logger.setLevel(logs_level)
pd.options.display.max_columns = None

2025-03-09 18:02:38,903 - DEBUG - rasterio.session - c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\rasterio\session.py - <module> - Could not import boto3, continuing with reduced functionality.
2025-03-09 18:02:38,905 - DEBUG - rasterio.env - c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\rasterio\env.py - <module> - GDAL data found in package: path='c:\\Users\\micka\\Projects\\13_potentiel_solaire\\algorithme\\.venv\\Lib\\site-packages\\rasterio\\gdal_data'.
2025-03-09 18:02:38,906 - DEBUG - rasterio.env - c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\rasterio\env.py - <module> - PROJ data found in package: path='c:\\Users\\micka\\Projects\\13_potentiel_solaire\\algorithme\\.venv\\Lib\\site-packages\\rasterio\\proj_data'.


# Extraction des données sources

### Etablissements scolaires

In [3]:
schools_establishments_path = extract_schools_establishments()
print(f"Annuaire des établissements scolaires extrait ici: {schools_establishments_path}")

Annuaire des établissements scolaires extrait ici: C:\Users\micka\Projects\13_potentiel_solaire\algorithme\data\fr-en-annuaire-education.geojson


### BD TOPO

In [4]:
bd_topo_path = extract_bd_topo(code_departement=code_departement)
print(f"BD TOPO extraite ici: {bd_topo_path}")

BD TOPO extraite ici: C:\Users\micka\Projects\13_potentiel_solaire\algorithme\data\BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15\BDTOPO\1_DONNEES_LIVRAISON_2024-12-00134\BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15\BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg


### BD PCI

In [5]:
bd_pci_path = extract_bd_pci(code_departement=code_departement)
print(f"BD PCI extraite ici: {bd_pci_path}")

BD PCI extraite ici: C:\Users\micka\Projects\13_potentiel_solaire\algorithme\data\PARCELLAIRE-EXPRESS_1-1__SHP_LAMB93_D093_2025-01-01\PARCELLAIRE-EXPRESS\1_DONNEES_LIVRAISON_2025-03-00057\PEPCI_1-1_SHP_LAMB93_D093\BATIMENT.SHP


### BD IRRADIATION

In [6]:
bd_irradiation_path = extract_bd_irradiation()
print(f"BD irradiation extraite ici: {bd_irradiation_path}")

BD irradiation extraite ici: C:\Users\micka\Projects\13_potentiel_solaire\algorithme\data\ENR_1-0_IRR-SOL_TIFF_WGS84G_FXX_2023-10-01\1_DONNEES_LIVRAISON\GlobalHorizontalIrradiation.tif


### BD Protected Buildings

In [7]:
bd_protected_buildings_path = extract_protected_buildings()
print(f"BD des bâtiments protégés extraite ici: {bd_protected_buildings_path}")

2025-03-09 18:03:05,523 - WARNING - pyogrio._io - c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\raw.py - read - Skipping field cog_insee_lors_de_la_protection: unsupported OGR type: 5
2025-03-09 18:03:05,524 - WARNING - pyogrio._io - c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\raw.py - read - Skipping field region: unsupported OGR type: 5
2025-03-09 18:03:05,525 - WARNING - pyogrio._io - c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\raw.py - read - Skipping field departement_en_lettres: unsupported OGR type: 5


BD des bâtiments protégés extraite ici: C:\Users\micka\Projects\13_potentiel_solaire\algorithme\data/liste_immeubles_proteges.geojson


# Filtre des données sur le périmètre du calcul

### Etablissements scolaires


In [8]:
schools_establishments = get_schools_establishments_of_interest(
    schools_filepath=schools_establishments_path,
    code_departement=code_departement,
    types_etablissements=['Ecole', 'Lycée', 'Collège'],
    statut_public_prive="Public",
    etat="OUVERT",
    crs=DEFAULT_CRS
)
nb_schools = schools_establishments.shape[0]
print(f"Nb d'établissements scolaires: {nb_schools}")

Nb d'établissements scolaires: 1130


### Zone d'intérêt géographique

In [9]:
codes_commune = schools_establishments["code_commune"].unique()
communes = gpd.read_file(bd_topo_path, layer="commune").to_crs(DEFAULT_CRS)
communes = communes[communes.code_insee.isin(codes_commune)]
geom_of_interest = communes.dissolve()[["geometry"]]

### Zones d'éducations

In [10]:
educational_zones = get_topo_zones_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    categories=["Science et enseignement"],
    natures=['Collège', 'Lycée', 'Enseignement primaire'],
    crs=DEFAULT_CRS
)
nb_educational_zones = educational_zones.shape[0]
print("Nb de zones d'éducations: ", nb_educational_zones)

c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\geopandas\io\file.py:497: UserWarning: More than one layer found in 'BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg': 'troncon_de_route' (default), 'route_n

Nb de zones d'éducations:  1088


### Bâtiments

In [11]:
# TODO : ajout des batiments manquants avec la BD PCI
buildings = get_topo_buildings_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    crs=DEFAULT_CRS
)
nb_buildings = buildings.shape[0]
print("Nb de batiments: ", nb_buildings)

c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_read_info(
c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\geopandas\io\file.py:497: UserWarning: More than one layer found in 'BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg': 'troncon_de_route' (default), 'route_n

Nb de batiments:  351576


### Zones avec des bâtiments protégés

In [12]:
areas_with_protected_buildings = get_areas_with_protected_buildings(
    bd_protected_buildings_path=bd_protected_buildings_path,
    geom_of_interest=geom_of_interest,
    crs=DEFAULT_CRS
)

# Détermination des bâtiments scolaires

In [13]:
schools_buildings = attach_buildings_to_schools(
    schools_establishments=schools_establishments,
    educational_zones=educational_zones,
    buildings=buildings
)
nb_schools_buildings = schools_buildings.shape[0]
print("Nb de batiments scolaires: ", nb_schools_buildings)

c:\Users\micka\Projects\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


Nb de batiments scolaires:  3731


# Calcul des attributs utiles pour le potentiel solaire

In [14]:
# TODO: v0 seulement à ce stade
solar_potential_of_schools_buildings = calculate_solar_potential(
    schools_buildings=schools_buildings,
    bd_irradiation_path=bd_irradiation_path,
    areas_with_protected_buildings=areas_with_protected_buildings
)

# Dump des donnees pour analyses

In [15]:
solar_potential_of_schools_buildings["zone_pour_rayonnement_solaire"] = solar_potential_of_schools_buildings["zone_pour_rayonnement_solaire"].to_wkt()

layers = ["schools_establishments", "educational_zones", "schools_buildings", "solar_potential_of_schools_buildings"]
gdfs = [schools_establishments, educational_zones, schools_buildings, solar_potential_of_schools_buildings]

for layer, gdf in zip(layers, gdfs):
    output_gpkg = DATA_FOLDER / f"{code_departement}_pipeline_results.gpkg"
    gdf.to_file(output_gpkg, layer=layer, driver="GPKG")

# Checks sur la qualité des données & calculs

In [16]:
nb_schools_with_buildings = len(schools_buildings.identifiant_de_l_etablissement.unique())
print("Nb d'établissements scolaires avec des batiments: {} ({}%)".format(
    nb_schools_with_buildings,
    round(100 * nb_schools_with_buildings / nb_schools)
))

Nb d'établissements scolaires avec des batiments: 445 (39%)


In [17]:
nb_buildings_protected = solar_potential_of_schools_buildings[solar_potential_of_schools_buildings.protection].shape[0]
print(f"Nb de batiments protégés {nb_buildings_protected} ({100*nb_buildings_protected/nb_schools_buildings:.2f}%)")

Nb de batiments protégés 1093 (29.30%)


# Aggrégations

### Par établissement scolaire

In [18]:
results_by_school = aggregate_solar_potential_by(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings,
    group_by = [
        "identifiant_de_l_etablissement",
    ]
)

results_by_school.head()

,identifiant_de_l_etablissement,surface_utile,rayonnement_solaire,potentiel_solaire,protection
0,0930043S,0.000000,0.000000,0.000000,False
1,0930089S,3245.297871,1143.404907,371066.369436,True
2,0930100D,2069.812266,1143.329590,236647.053457,False
3,0930116W,0.000000,0.000000,0.000000,False
4,0930117X,4907.794034,1141.873413,560407.952443,True


### Par commune

In [19]:
results_by_city = aggregate_solar_potential_by(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings,
    group_by = [
        "code_commune",
        "nom_commune",
        "code_departement",
        "libelle_departement",
        "code_region",
        "libelle_region",
    ]
)

results_by_city.head()

,code_commune,nom_commune,code_departement,libelle_departement,code_region,libelle_region,surface_utile,rayonnement_solaire,potentiel_solaire,protection
0,93001,Aubervilliers,093,Seine-Saint-Denis,11,Ile-de-France,54563.526631,967.649109,6.231777e+06,True
1,93005,Aulnay-sous-Bois,093,Seine-Saint-Denis,11,Ile-de-France,35282.028352,918.783203,4.040277e+06,True
2,93006,Bagnolet,093,Seine-Saint-Denis,11,Ile-de-France,25830.017170,1027.417969,2.955298e+06,True
3,93007,Le Blanc-Mesnil,093,Seine-Saint-Denis,11,Ile-de-France,29601.228822,915.268311,3.387507e+06,True
4,93008,Bobigny,093,Seine-Saint-Denis,11,Ile-de-France,28739.882571,876.470520,3.282204e+06,True


### Pour le département

In [20]:
results_by_departement = aggregate_solar_potential_by(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings,
    group_by = [
        "code_departement",
        "libelle_departement",
        "code_region",
        "libelle_region",
    ]
)

results_by_departement

,code_departement,libelle_departement,code_region,libelle_region,surface_utile,rayonnement_solaire,potentiel_solaire,protection
0,093,Seine-Saint-Denis,11,Ile-de-France,867308.706529,965.961365,9.916325e+07,True


# Sauvegarde des calculs en Base de Données

In [21]:
conn = duckdb.connect('./../database/potentiel_solaire.duckdb')

### Mise à jour des données des Etablissements du Département en Base de Données

In [ ]:
conn.register("results_by_school", results_by_school)

query = """
UPDATE etablissements
SET 
    surface_utile = results_by_school.surface_utile,
    rayonnement_solaire = results_by_school.rayonnement_solaire,
    potentiel_solaire = results_by_school.potentiel_solaire,
    protection = results_by_school.protection
FROM
    results_by_school
WHERE
    etablissements.identifiant_de_l_etablissement = results_by_school.identifiant_de_l_etablissement
"""

conn.execute(query)
conn.commit()

assert conn.execute("SELECT COUNT(*) FROM etablissements WHERE surface_utile > 0").fetchone()[0] > 0, "Problème lors de la mise à jour des établissements"

❔


### Mise à jour des données des Communes du Département en Base de Données

In [ ]:
query = f"""
UPDATE communes
SET 
    surface_utile = agg.surface_utile,
    potentiel_solaire = agg.potentiel_solaire,
    count_etablissements = agg.count_etablissements,
    count_etablissements_proteges = agg.count_etablissements_proteges
FROM (
    SELECT 
         code_commune,
         SUM(surface_utile) AS surface_utile,
         SUM(potentiel_solaire) AS potentiel_solaire,
         COUNT(*) AS count_etablissements,
         SUM(CASE WHEN protection THEN 1 ELSE 0 END) AS count_etablissements_proteges
    FROM etablissements
    GROUP BY code_commune
) AS agg
WHERE
  communes.code_commune = agg.code_commune
  AND communes.code_departement = '{code_departement}'
"""

conn.execute(query)
conn.commit()

assert conn.execute("SELECT COUNT(*) FROM communes WHERE surface_utile > 0").fetchone()[0] > 0, "Problème lors de la mise à jour des communes"

### Mise à jour des données du Département en Base de données

In [40]:
query = f"""
UPDATE departements
SET 
    surface_utile = agg.surface_utile,
    potentiel_solaire = agg.potentiel_solaire,
    count_etablissements = agg.count_etablissements,
    count_etablissements_proteges = agg.count_etablissements_proteges
FROM (
    SELECT 
         code_departement,
         SUM(surface_utile) AS surface_utile,
         SUM(potentiel_solaire) AS potentiel_solaire,
         COUNT(*) AS count_etablissements,
         SUM(CASE WHEN protection THEN 1 ELSE 0 END) AS count_etablissements_proteges
    FROM etablissements
    GROUP BY code_departement
) AS agg
WHERE
    departements.code_departement = agg.code_departement
    AND departements.code_departement = '{code_departement}'
"""

conn.execute(query)
conn.commit()

assert conn.execute(f"SELECT COUNT(*) FROM departements WHERE surface_utile > 0 AND code_departement = '{code_departement}'").fetchone()[0] > 0, f"Problème lors de la mise à jour du département {code_departement}"

# Export sous forme de fichiers GeoJSON

In [23]:
output_folder = ALGORITHME_FOLDER.parent / "results" / f"D{code_departement}"
output_folder.mkdir(exist_ok=True, parents=True)

# TODO : a voir cote front sil faut changer le format de mise a disposition
# TODO2 : créer les fichiers GeoJSON à partir de la Base de données dans un autre pipeline
#results_by_school.to_file(output_folder / f"D{code_departement}_ecoles.geojson", driver="GeoJSON")
#results_by_commune.to_file(output_folder / f"D{code_departement}_communes.geojson", driver="GeoJSON")
#results_by_departement.to_file(output_folder / f"D{code_departement}_departement.geojson", driver="GeoJSON")